In [ ]:
# !rm -rf /content/hop_length

# **Installation**

In [1]:
!pip install audb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.1/241.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.5 MB/s eta 0:00:00


In [2]:
!pip install pydub

In [3]:
!pip install shap

In [4]:
!pip install opensmile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.0/996.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 5.2 MB/s eta 0:00:00


In [5]:
# !pip install alibi

In [6]:
import os
import time
import datetime
import errno
import random
import copy
import pickle
from tensorflow.keras.optimizers import Adam
import joblib
import collections
from datetime import timedelta, date
from collections import Counter
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
import joblib
import audb
import audiofile
import opensmile
import shap
import sklearn
from sklearn.utils import shuffle
from sklearn.datasets import make_classification
from sklearn.model_selection import (
    train_test_split, KFold, StratifiedKFold, ParameterGrid
)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, auc, log_loss, confusion_matrix,
    classification_report, precision_recall_curve
)
from tensorflow.keras.regularizers import l2
from imblearn.over_sampling import SMOTE
import imblearn

from xgboost import XGBClassifier

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import (
    Dense, Activation, LSTM, Bidirectional, RepeatVector, Dropout,
    TimeDistributed, Conv1D, MaxPooling1D, Flatten, ConvLSTM2D
)
from pydub import AudioSegment
from pydub.silence import detect_nonsilent

# Seaborn theme setup
sns.set_theme()

# Inline plotting for Jupyter Notebook
%matplotlib inline


# **Documentation**


Documentation opensmile : https://audeering.github.io/opensmile-python/install.html

Github : https://audeering.github.io/opensmile-python/install.html

Audiofile : https://audeering.github.io/audiofile/installation.html

# **LowLevelDescriptors**

In [7]:
# Hyper Paramaters

#LSTM
univariate = False
n_steps_in = 10
n_seq = 2
n_steps_out = 2
n_features_in = 65
n_features_out = 2
hidden_layers = 50
Epochs = 1000
No_exp = 1
batch_size = 128

print(f"Univariate: {univariate}")
print(f"Number of Input Time Steps: {n_steps_in}")
print(f"Number of Input Sequences: {n_seq}")
print(f"Number of Output Time Steps: {n_steps_out}")
print(f"Number of Input Features: {n_features_in}")
print(f"Number of Output Features: {n_features_out}")
print(f"Number of Hidden Layers: {hidden_layers}")
print(f"Epochs: {Epochs}")
print(f"Number of Experiments: {No_exp}")
print(f"Batch size: {batch_size}")

#Optimiser
lr_adam = 0.0001

#Loss Function
label_smoothing = 0.05

#Activation
negative_slope = 0.01

#Optimizer and Loss Function
optimizer = Adam(learning_rate = lr_adam)
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = label_smoothing)
activation_lstm = tf.keras.layers.LeakyReLU(negative_slope = negative_slope)
# activation_lstm = tf.keras.activations.swish
# activation_lstm = 'relu'
activation_dense = 'softmax'

feature_list = ['F0final_sma',
 'voicingFinalUnclipped_sma',
 'jitterLocal_sma',
 'jitterDDP_sma',
 'shimmerLocal_sma',
 'logHNR_sma',
 'audspec_lengthL1norm_sma',
 'audspecRasta_lengthL1norm_sma',
 'pcm_RMSenergy_sma',
 'pcm_zcr_sma',
 'audSpec_Rfilt_sma[0]',
 'audSpec_Rfilt_sma[1]',
 'audSpec_Rfilt_sma[2]',
 'audSpec_Rfilt_sma[3]',
 'audSpec_Rfilt_sma[4]',
 'audSpec_Rfilt_sma[5]',
 'audSpec_Rfilt_sma[6]',
 'audSpec_Rfilt_sma[7]',
 'audSpec_Rfilt_sma[8]',
 'audSpec_Rfilt_sma[9]',
 'audSpec_Rfilt_sma[10]',
 'audSpec_Rfilt_sma[11]',
 'audSpec_Rfilt_sma[12]',
 'audSpec_Rfilt_sma[13]',
 'audSpec_Rfilt_sma[14]',
 'audSpec_Rfilt_sma[15]',
 'audSpec_Rfilt_sma[16]',
 'audSpec_Rfilt_sma[17]',
 'audSpec_Rfilt_sma[18]',
 'audSpec_Rfilt_sma[19]',
 'audSpec_Rfilt_sma[20]',
 'audSpec_Rfilt_sma[21]',
 'audSpec_Rfilt_sma[22]',
 'audSpec_Rfilt_sma[23]',
 'audSpec_Rfilt_sma[24]',
 'audSpec_Rfilt_sma[25]',
 'pcm_fftMag_fband250-650_sma',
 'pcm_fftMag_fband1000-4000_sma',
 'pcm_fftMag_spectralRollOff25.0_sma',
 'pcm_fftMag_spectralRollOff50.0_sma',
 'pcm_fftMag_spectralRollOff75.0_sma',
 'pcm_fftMag_spectralRollOff90.0_sma',
 'pcm_fftMag_spectralFlux_sma',
 'pcm_fftMag_spectralCentroid_sma',
 'pcm_fftMag_spectralEntropy_sma',
 'pcm_fftMag_spectralVariance_sma',
 'pcm_fftMag_spectralSkewness_sma',
 'pcm_fftMag_spectralKurtosis_sma',
 'pcm_fftMag_spectralSlope_sma',
 'pcm_fftMag_psySharpness_sma',
 'pcm_fftMag_spectralHarmonicity_sma',
 'mfcc_sma[1]',
 'mfcc_sma[2]',
 'mfcc_sma[3]',
 'mfcc_sma[4]',
 'mfcc_sma[5]',
 'mfcc_sma[6]',
 'mfcc_sma[7]',
 'mfcc_sma[8]',
 'mfcc_sma[9]',
 'mfcc_sma[10]',
 'mfcc_sma[11]',
 'mfcc_sma[12]',
 'mfcc_sma[13]',
 'mfcc_sma[14]']

Univariate: False
Number of Input Time Steps: 10
Number of Input Sequences: 2
Number of Output Time Steps: 2
Number of Input Features: 65
Number of Output Features: 2
Number of Hidden Layers: 50
Epochs: 1000
Number of Experiments: 1
Batch size: 128


In [8]:
def preprocess_(folder_path, label, smile, n=10):
    X, y = [], []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        if not file_path.lower().endswith(('.wav', '.mp3', '.flac')):
            continue

        try:
            signal, sampling_rate = audiofile.read(file_path, always_2d=True)
            features = smile.process_signal(signal, sampling_rate).to_numpy()

            segments = [features[i:i+n] for i in range(0, len(features) - n + 1, n)]
            segments = [segment for segment in segments if segment.shape == (n, features.shape[1])]

            if not segments:
                continue

            X.extend(segments)
            y.extend([label] * len(segments))

        except Exception as e:
            print(f"Error processing {file_name}: {e}")

    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int32)

In [9]:
def evaluate_model_(parkinson_path, healthy_path, model_path, n=10, scaler_path=None, scale_option=None, save=True):
    smile = opensmile.Smile(
        feature_set=opensmile.FeatureSet.ComParE_2016,
        feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
    )

    model = load_model(model_path, custom_objects={'LeakyReLU': activation_lstm})
    model.summary()

    X_parkinson, y_parkinson = preprocess_(parkinson_path, 1, smile, n)
    X_healthy, y_healthy = preprocess_(healthy_path, 0, smile, n)

    min_size = min(len(y_parkinson), len(y_healthy))

    indices_parkinson = np.random.choice(len(y_parkinson), min_size, replace=False)
    indices_healthy = np.random.choice(len(y_healthy), min_size, replace=False)

    X_parkinson, y_parkinson = X_parkinson[indices_parkinson], y_parkinson[indices_parkinson]
    X_healthy, y_healthy = X_healthy[indices_healthy], y_healthy[indices_healthy]

    X = np.vstack((X_parkinson, X_healthy))
    y = np.hstack((y_parkinson, y_healthy))
    indices = np.random.permutation(len(y))
    X, y = X[indices], y[indices]

    X_reshaped = X.reshape(-1, X.shape[-1])

    if scale_option == 'preloaded':
        scaler = joblib.load(scaler_path)
    else:
        scaler = MinMaxScaler()
        scaler.fit(X_reshaped)

    X_scaled = scaler.transform(X_reshaped)
    X_normalized = X_scaled.reshape(X.shape)


    # ----- cnn-lstm
    # X_normalized = np.asarray(X_normalized)
    # X_normalized = X_normalized.reshape((X_normalized.shape[0], n_seq, int(n_steps_in/n_seq), n_features_in))

    #----- conv-lstm
    # X_normalized = np.asarray(X_normalized)
    # X_normalized = X_normalized.reshape((X_normalized.shape[0], n_seq, 1, int(n_steps_in/n_seq), n_features_in))

    x_train = X_normalized

    print("Model Input Shape:", model.input_shape)
    print("Data Shape:", X_normalized.shape)

    explainer = shap.GradientExplainer(model, x_train)
    shap_values = explainer.shap_values(x_train)
    shap_feature_importance = np.mean(np.abs(shap_values), axis=(0, 1, -1))
    df_importance = pd.DataFrame({
      'Feature': feature_list,
      'SHAP Importance': shap_feature_importance
    })

    df_importance = df_importance.sort_values(by="SHAP Importance", ascending=False)

    if save:
      df_importance.to_csv("shap_feature_importance.csv", index=False)

    return df_importance

In [11]:
parkinson_path = '/content/PwPD'
healthy_path = '/content/HS'
model_path = '/content/model_vanilla_italian_a.h5'
evaluate_model_(parkinson_path, healthy_path, model_path, n=10, scaler_path=None, scale_option=None, save=True)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 50)                  │          23,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             102 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,304 (91.04 KB)

 Trainable params: 23,302 (91.02 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Model Input Shape: (None, 10, 65)
Data Shape: (506, 10, 65)


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer_3
Received: inputs=['Tensor(shape=(506, 10, 65))']
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer_3
Received: inputs=['Tensor(shape=(50, 10, 65))']
  warnings.warn(msg)


,Feature,SHAP Importance
9,pcm_zcr_sma,0.020453
38,pcm_fftMag_spectralRollOff25.0_sma,0.006890
6,audspec_lengthL1norm_sma,0.006720
57,mfcc_sma[7],0.005979
52,mfcc_sma[2],0.005628
...,...,...
29,audSpec_Rfilt_sma[19],0.000031
12,audSpec_Rfilt_sma[2],0.000024
15,audSpec_Rfilt_sma[5],0.000020
13,audSpec_Rfilt_sma[3],0.000019
